In [10]:
# 

In [2]:
import os
import time
import json
import requests
from openai import AzureOpenAI
from dotenv import load_dotenv
from datetime import datetime, date
from zoneinfo import ZoneInfo
from openai.types.beta import Thread
from openai.types.beta import Assistant


load_dotenv()
# Initialize the Azure OpenAI client
assistant_client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version=os.getenv('AZURE_OPENAI_API_VERSION')
)

# Define the deployment you want to use for your chat completions API calls

deployment_name = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
# AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED

In [3]:
# %pip install --upgrade azure-ai-inference

In [4]:
# from azure.core.settings import settings

# settings.tracing_implementation = "opentelemetry"

In [5]:
from azure.ai.inference.tracing import AIInferenceInstrumentor
# Instrument AI Inference API 

AIInferenceInstrumentor().instrument()

In [6]:
# AIInferenceInstrumentor().uninstrument() 

In [7]:
# %pip install azure-monitor-opentelemetry

In [8]:
# import os
# from azure.monitor.opentelemetry import configure_azure_monitor

# os.environ['AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED'] = 'true'
# # Enable Azure Monitor tracing
# application_insights_connection_string = project.telemetry.get_connection_string()
# if not application_insights_connection_string:
#     print("Application Insights was not enabled for this project.")
#     print("Enable it via the 'Tracing' tab in your AI Studio project page.")
#     exit()
    
# configure_azure_monitor(connection_string=application_insights_connection_string)

In [9]:
name_date = "date_time_assistant"
instructions_date = """You are analytics agent that obtains today's date and time in the local timezone of the user."""
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_date",
            "description": "Get the current date and time in the local timezone of the user.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    }
]

verbose_output = True


In [10]:
name_schema = "data_schema_assistant"
instructions_schema = """"You are a data analyst that analyzes the table metadata and returns the schema."""
tools = [
    {
        "type": "function",
        "function": {
            "name": "generate_table_schema",
            "description": "analyzes the table metadata and returns the schema",
            "parameters": {"type": "object", "properties": {}, "required": []},
        },
    }
]

verbose_output = True

In [11]:
def get_current_date():
    """Get the current date"""
    now = datetime.now()
    # Print the current time
    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)
    # Get the current date
    today = date.today()
    # Format the date
    formatted_date = today.strftime("%B %d, %Y")
    # Print the formatted date
    print("Today's date:", formatted_date)

    return json.dumps({"current_time": current_time, "current_date": formatted_date})

In [12]:
def generate_table_schema(url):
    url = "http://roapi-app-arash.dhbjd9bvccbjedea.eastus.azurecontainer.io:8000/api/schema"
    response = requests.get(url)
    if response.status_code == 200:
        schema = response.json()
        print(schema)
    else:
        print("Error:", response.status_code)
    # Sample JSON output
    result = []  
    for table_name, table_info in schema.items():  
        columns = []  
        for field in table_info['fields']:  
            column_name = field['name']  
            data_type = field['data_type']  
              
            # Map the data type to the desired format  
            if data_type == 'Utf8':  
                sql_data_type = 'VARCHAR'  
            elif data_type == 'Int64' or data_type == 'Int32':  
                sql_data_type = 'INTEGER'  
            elif data_type == 'Boolean':  
                sql_data_type = 'BOOLEAN'  
            else:  
                sql_data_type = 'UNKNOWN'  # Handle any unexpected data types  
              
            columns.append(f"{column_name} ({sql_data_type})")  
          
        columns_str = ", ".join(columns)  
        result.append(f"Table '{table_name}' with columns: {columns_str}")  
      
    return "\n".join(result)

In [13]:
current_date = get_current_date()

Current Time = 15:16:57
Today's date: November 21, 2024


In [14]:
url = "http://roapi-app-arash.dhbjd9bvccbjedea.eastus.azurecontainer.io:8000/api/schema"
table_schema = generate_table_schema(url)
generate_table_schema(url)

{'asl': {'fields': [{'name': 'Id', 'data_type': 'Utf8', 'nullable': True, 'dict_id': 0, 'dict_is_ordered': False, 'metadata': {}}, {'name': 'AccountId', 'data_type': 'Utf8', 'nullable': True, 'dict_id': 0, 'dict_is_ordered': False, 'metadata': {}}, {'name': 'AccountType', 'data_type': 'Utf8', 'nullable': True, 'dict_id': 0, 'dict_is_ordered': False, 'metadata': {}}, {'name': 'DeviceId', 'data_type': 'Utf8', 'nullable': True, 'dict_id': 0, 'dict_is_ordered': False, 'metadata': {}}, {'name': 'DeviceType', 'data_type': 'Utf8', 'nullable': True, 'dict_id': 0, 'dict_is_ordered': False, 'metadata': {}}, {'name': 'ApiName', 'data_type': 'Utf8', 'nullable': True, 'dict_id': 0, 'dict_is_ordered': False, 'metadata': {}}, {'name': 'LoggedTime', 'data_type': 'Utf8', 'nullable': True, 'dict_id': 0, 'dict_is_ordered': False, 'metadata': {}}, {'name': 'ProfileId', 'data_type': 'Utf8', 'nullable': True, 'dict_id': 0, 'dict_is_ordered': False, 'metadata': {}}, {'name': 'TenantId', 'data_type': 'Utf8', 

"Table 'asl' with columns: Id (VARCHAR), AccountId (VARCHAR), AccountType (VARCHAR), DeviceId (VARCHAR), DeviceType (VARCHAR), ApiName (VARCHAR), LoggedTime (VARCHAR), ProfileId (VARCHAR), TenantId (VARCHAR), MediaFirstRequiredSku (VARCHAR), DvrCapability (VARCHAR), date (VARCHAR)\nTable 'bitmovin' with columns: ad (VARCHAR), analytics_version (VARCHAR), asn (VARCHAR), audio_bitrate (INTEGER), audio_language (VARCHAR), browser (VARCHAR), browser_is_bot (BOOLEAN), browser_version_major (VARCHAR), browser_version_minor (VARCHAR), buffered (INTEGER), cdn_provider (VARCHAR), city (VARCHAR), client_time (INTEGER), country (VARCHAR), custom_user_id (VARCHAR), date (VARCHAR), day (INTEGER), device_class (VARCHAR), device_type (VARCHAR), domain (VARCHAR), drm_type (VARCHAR), dropped_frames (INTEGER), duration (INTEGER), hour (INTEGER), impression_id (VARCHAR), ip_address (VARCHAR), is_casting (BOOLEAN), is_live (BOOLEAN), is_muted (BOOLEAN), isp (VARCHAR), language (VARCHAR), m3u8_url (VARCHAR

In [15]:
name_sql = "sql_query_assistant"
instructions_sql = """You are a data analyst that generates SQL query that conforms to PostressSQL style analysis using table metadata and user query."""
tools = [
    {
        "type": "function",
        "function": {
            "name": "execute_sql_query",
            "description": "Given table metadata and user qurey performs SQL anaylis",
            "parameters": {"type": "object", "properties": {}, "required": ["current_date", "table_schema", "user_query"]},
        },
    }
]

verbose_output = True

In [18]:
def execute_sql_query(current_date, table_schema, user_query) -> str:
    import requests
    """Execute the latest SQL queries."""
    data_sql_request = assistant_client.chat.completions.create(
        model=deployment_name,
        messages=[
            {
                "role": "user",
                "content": f"""
                You are an expert at ROAPI SQL analysis. You have access to ROAPI with the table descriptions.
                Given a user question related to the data in the database, generate the SQL query that conforms to Postgres SQL style. Distinct the columns that need to be in double quotes and those that not.
                Below are some examples of the SQL queries:
                1- SELECT COUNT(DISTINCT user_id) AS active_user_count FROM bitmovin WHERE date >= to_date(cast(now() AS VARCHAR)) - INTERVAL '200 days' AND user_id IS NOT NULL;
                2- SELECT * FROM bitmovin WHERE date >= to_date(cast(now() AS VARCHAR)) - INTERVAL '200 days' AND user_id IS NOT NULL LIMIT 10;
                3- SELECT "AccountType", "DeviceType", "ApiName", COUNT(*) AS api_usage_count FROM asl GROUP BY "AccountType", "DeviceType", "ApiName" ORDER BY "AccountType", "DeviceType", api_usage_count DESC;
                4- SELECT asl."TenantId", asl."DeviceType", bitmovin."browser", COUNT(*) AS usage_count, AVG(bitmovin."page_load_time") AS avg_page_load_time, AVG(bitmovin."startuptime") AS avg_startup_time FROM asl JOIN bitmovin ON asl."TenantId" = bitmovin."tenant" GROUP BY asl."TenantId", asl."DeviceType", bitmovin."browser" ORDER BY usage_count DESC LIMIT 10;
                Make sure to make date queries align to today's date.
                Only give the SQL query in the response, nothing else, don't add sql in front. Just SQL query.
                {current_date}
                {table_schema}
                {user_query}
                """
            }
        ],
        max_tokens=4096,
    )
    sql_query = data_sql_request.choices[0].message.content
    print('This is the sql query:', sql_query)
    # Execute SQL query
    url = "http://roapi-app-arash.dhbjd9bvccbjedea.eastus.azurecontainer.io:8000/api/sql"
    response = requests.post(url, data=sql_query)

    # Check if the request was successful
    data_sql = None  # Initialize data_sql

    if response.status_code == 200:
        try:
            data_sql = response.json()
            print(data_sql)
        except requests.JSONDecodeError:
            print("Failed to decode JSON response.")
    else:
        print(f"Failed to execute SQL query. Status code: {response.status_code}")
        data_sql = [] 

    return data_sql

In [21]:
execute_sql_query(current_date, table_schema, '5 records from asl')

This is the sql query: SELECT COUNT(DISTINCT "user_id") AS active_user_count 
FROM bitmovin 
WHERE date >= to_date('2024-11-21', 'YYYY-MM-DD') - INTERVAL '200 days' 
AND "user_id" IS NOT NULL;
Failed to execute SQL query. Status code: 400


[]

In [50]:
name_pa = "user_proxy"
agent_arr = ["date_time_assistant", "data_schema_assistant", "sql_query_assistant"]
agent_string = ""
for item in agent_arr:
    agent_string += f"{item}\n"

instructions_pa = f"""As a user proxy agent, your primary function is to streamline dialogue between the user and the specialized agents within this group chat. You are tasked with articulating user inquiries with clarity to the relevant agents and maintaining a steady flow of communication to guarantee the user's request is comprehensively addressed. Please withhold your response to the user until the task is completed, unless an issue is flagged by the respective agent or when you can provide a conclusive reply.
You can ask data_schema_assistant to determine the schema and send it to the sql_query_assistant to query based on user input for analysis.

You have access to the following agents to accomplish the task:
{agent_string}
If the agents above are not enough or are out of scope to complete the task, then run send_message with the name of the agent.

When outputting the agent names, use them as the basis of the agent_name in the send message function, even if the agent doesn't exist yet.

Run the send_message function for each agent name generated. 

Do not ask for followup questions, run the send_message function according to your initial input.

Plan:
1. date_time_assistant gets current date to perform SQL query datastamp analysis
2. data_schema_assistant generates table information to provide to sql_query_assistant
3. sql_query_assistant creates a SQL query based on user input and table schema and executes it to answer user question

Now take a deep breath and accomplish the plan above. Always follow the plan step by step in the exact order and do not ask for followup questions. Do not skip any steps in the plan, do not repeat any steps and always complete the entire plan in order step by step.  
The data_schema_assistant and date_time_assistant can never run more than one time in a row, review your plan before running the next step.
"""

tools = [
    {"type": "code_interpreter"},
    {
        "type": "function",
        "function": {
            "name": "send_message",
            "description": "Send messages to other agents in this group chat.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The message to be sent",
                    },
                    "agent_name": {
                        "type": "string",
                        "description": "The name of the agent to execute the task.",
                    },
                },
                "required": ["query", "agent_name"],
            },
        },
    },
]

verbose_output = True

In [51]:
user_proxy = assistant_client.beta.assistants.create(
  name=name_pa,
  instructions=instructions_pa,
  model=deployment_name,
  tools=tools
)
date_time_assistant = assistant_client.beta.assistants.create(
  name=name_date,
  instructions=instructions_date,
  model=deployment_name,
  tools=tools
)
data_schema_assistant = assistant_client.beta.assistants.create(
    name=name_schema,
    instructions=instructions_schema,
    model=deployment_name,
    tools=tools
)
sql_query_assistant = assistant_client.beta.assistants.create(
    name=name_sql,
    instructions=instructions_sql,
    model=deployment_name,
    tools=tools
)

In [52]:
from typing import Dict, Optional

agents_threads: Dict[str, Dict[str, Optional[str]]] = {
    "date_time_assistant": {"agent": date_time_assistant, "thread": None},
    "data_schema_assistant": {"agent": data_schema_assistant, "thread": None},
    "sql_query_assistant": {"agent": sql_query_assistant, "thread": None}
}


# Define the send_message function with only the query parameter
def send_message(query: str, agent_name: str) -> str:
    # Check if the agent_name is in agents_threads
    if agent_name not in agent_arr:
        print(
            f"Agent '{agent_name}' does not exist. This means that the multi-agent system does not have the necessary agent to execute the task. *** FUTURE CODE: AGENT SWARM***"
        )
        # return None
    # If the program has not exited, proceed with setting the agent recipient
    recipient_type = agent_name
    recipient_info = agents_threads[recipient_type]

    # If the program has not exited, proceed with setting the agent recipient
    recipient_type = agent_name
    recipient_info = agents_threads[recipient_type]

    # Create a new thread if user proxy and agent thread does not exist
    if not recipient_info["thread"]:
        thread_object = assistant_client.beta.threads.create()
        recipient_info["thread"] = thread_object

    # This function dispatches a message to the proper agent and it's thread
    return dispatch_message(query, recipient_info["agent"], recipient_info["thread"])
    # print("Proxy Assistant Message: " + message)

In [53]:
def dispatch_message(message: str, agent: Assistant, thread: Thread) -> str:
    # Loops through all the agents functions to determine which function to use

    available_functions = {}
    # Iterate through each tool in the agent.tools list
    for tool in agent.tools:
        # Check if the tool has a 'function' attribute
        if hasattr(tool, "function"):
            function_name = tool.function.name
            # Attempt to retrieve the function by its name and add it to the available_functions dictionary
            if function_name in globals():
                available_functions[function_name] = globals()[function_name]
        else:
            # Handle the case where the tool does not have a 'function' attribute
            print("This tool does not have a 'function' attribute.")
    # Draft a new message as part of the ongoing conversation.
    message = assistant_client.beta.threads.messages.create(thread_id=thread.id, role="user", content=message)
    # Carry out the tasks delineated in this discussion thread.
    run = assistant_client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=agent.id,
    )
    while True:
        # Await the completion of the thread execution.
        while run.status in ["queued", "in_progress"]:
            run = assistant_client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
            time.sleep(1)

        # If an action is necessary, initiate the appropriate function to perform the task.
        if run.status == "requires_action":
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            for _tool_call in tool_calls:
                tool_responses = []
                if (
                    run.required_action.type == "submit_tool_outputs"
                    and run.required_action.submit_tool_outputs.tool_calls is not None
                ):
                    tool_calls = run.required_action.submit_tool_outputs.tool_calls
                    for call in tool_calls:
                        if call.type == "function":
                            if call.function.name not in available_functions:
                                raise Exception("Function requested by the model does not exist")

                            # Assign the appropriate function to the agent for invocation.
                            function_to_call = available_functions[call.function.name]
                            tool_response = function_to_call(**json.loads(call.function.arguments))
                            tool_responses.append({"tool_call_id": call.id, "output": tool_response})

            # Present the outcomes produced by the tool.
            run = assistant_client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id, run_id=run.id, tool_outputs=tool_responses
            )

        # if the run is completed, return the assistant message else provide error
        elif run.status == "failed":
            raise Exception("Run Failed. ", run.last_error)
        # Craft a reply from the assistant.
        else:
            messages = assistant_client.beta.threads.messages.list(thread_id=thread.id)

            # Transmit the response message back to the facilitating agent.
            return messages.data[0].content[0].text.value

In [ ]:
thread = assistant_client.beta.threads.create()
# Initiate proxy agent and the main thread. This thread will remain active until the task is completed and will serve as the main communication thread between the other agents.
user_message = input("User Query: Count of active users in last 200 days")
message = dispatch_message(user_message, user_proxy, thread)

In [ ]:
# # Create a new thread if user proxy and agent thread does not exist
# if not recipient_info["thread"]:a
#       thread_object = assistant_client.beta.threads.create()
#       recipient_info["thread"] = thread_object